download google takeout?



In [ ]:
var importer = require('../Core');

var listTakeouts = () => {
    return client
    .url('https://takeout.google.com/settings/takeout')
    .loginGoogle()
    .waitUntil(() => client.getUrl()
           .then(url => url.indexOf('privacy') > -1 || url.indexOf('takeout') > -1), 20000, '')
    .url('https://takeout.google.com/settings/takeout')
    .getAllXPath([
        '//*[@data-id]',
        {
            id: './@data-id',
            label: './/td[3]//text()'
        }
    ])
};

var takeoutProducts = (product) => {
    return client
    .url('https://takeout.google.com/settings/takeout/custom/' + product.id)
    .pause(1000)
    .click('//*[contains(@role, "button")][contains(., "Next")]')
    .pause(1000)
    .execute(() => {
        var el = document.evaluate(
            '//*[contains(., "Archive size")]/parent::*//*[contains(@role,"option")][contains(.,"2")]',
            document,
            null,
            XPathResult.FIRST_ORDERED_NODE_TYPE, null
        ).singleNodeValue;
        var event = document.createEvent('MouseEvents');
        event.initEvent('mousedown', true, true);
        el.dispatchEvent(event, true);
    })
    .pause(1000)
    // TODO: shorten this in to a utility command
    // doesn't work because of some weird css BS 
    // .click('//*[contains(@role, "option")][contains(., "50")]')
    .click('//*[contains(., "Archive size")]/parent::*//*[contains(@role,"listbox")]/div[3]//*[contains(@role,"option")][contains(.,"50")]')
    .click('//*[contains(@role, "button")][contains(., "Create archive")]')
    .waitUntil(() => client.isExisting('//td[last()]//*[contains(@data-download-uri, "takeout/download")]'), 90000, '')
    .click('//td[last()]//*[contains(@data-download-uri, "takeout/download")]')
    .loginGoogle()
};

var downloadGoogleTakeout = (products) => {
    var regex = new RegExp(products, 'ig');
    var selectedProducts;
    return listTakeouts()
    .then(ids => (selectedProducts = ids
        .map(k => ({
            id: k.id.join(' '),
            label: k.label.join(' ')
        }))
        .filter(k => k.id.match(regex) || k.label.match(regex))))
    .then(() => {
        console.log(selectedProducts);
        return importer.runAllPromises(selectedProducts
        .map(p => (resolve) => {
            return takeoutProducts(p)
            .then(r => resolve(r))
            .catch(e => resolve(e))
        }))
    })
};
if (typeof client.downloadGoogleTakeout == 'undefined') {
    client.addCommand('downloadGoogleTakeout', downloadGoogleTakeout);
}
module.exports = downloadGoogleTakeout;
downloadGoogleTakeout;



order google takeout?



In [2]:
var importer = require('../Core');

var runSeleniumCell, downloadGoogleTakeout;
var googleTakeout = (products = 'all?') => {
    return importer.interpretAll(['selenium cell'])
    .then(r => {
        runSeleniumCell = r[0].runInNewContext({});
        return runSeleniumCell([
            'log in google',
            'get all elements',
            'download google takeout'
        ]);
    })
    .then(r => {
        downloadGoogleTakeout = r[2];
        return downloadGoogleTakeout(products);
    })
    .then(r => {
        // TODO: save it in the project path?
        return r;
    })
};
module.exports = googleTakeout;
googleTakeout;



[Function: googleTakeout]

test google takeout



In [48]:
var importer = require('../Core');

$$.async();
var googleTakeout;
importer.interpretAll(['order google takeout'])
.then(r => {
    googleTakeout = r[0].runInNewContext();
    return googleTakeout('location history');
})
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
[ [Function: loginGoogle],
  [Function: getAllXPath],
  [Function: downloadGoogleTakeout] ]
[ { id: 'location_history', label: 'Location History' } ]
Google: Require password


[ { sessionId: 'c116a113a13b038009c5e07f55d32e47',
    status: 0,
    value: null } ]

d3 stream my chrome history by category and site sub-divided?

